In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
students = pd.DataFrame([1,1])

def selectData(students: pd.DataFrame) -> pd.DataFrame:
    return students.shape

selectData(students)

(2, 1)

In [3]:
print(selectData(students))

(2, 1)


In [4]:
def createBonusColumn(students: pd.DataFrame) -> pd.DataFrame:
    students['bonus'] = students['salary'] * 2
    return students

In [5]:
conn = sqlite3.connect(':memory:')
conn1 = sqlite3.connect('db.db')

In [6]:
cursor = conn.cursor()

In [7]:
cursor.execute("""
    INSERT INTO students (name, class)
    VALUES (?, ?)
""", ("Alice", "Math"))

OperationalError: no such table: students

In [ ]:
cursor.execute("""PRAGMA table_info(students)""")

In [ ]:
print(cursor.fetchall())

[(0, 'id', '', 0, None, 1), (1, 'name', 'VARCHAR', 0, None, 0), (2, 'class', 'VARCHAR', 0, None, 0)]


In [ ]:
students = pd.DataFrame([(1,1),(2,1)])

In [ ]:
selectData(students)

(2, 2)

In [ ]:
students = pd.DataFrame([[1,1],[2,2],[3,3]])

In [41]:
cursor.execute("""
                        SELECT c.customer_id, c.customer_name, SUM(o.total_amount) AS total_sales_2024
                        FROM Customer c
                        JOIN Orders o
                        ON o.customer_id = c.customer_id
                        WHERE o.order_date BETWEEN '2024-01-01' AND '2025-01-01'
                        GROUP BY c.customer_id, c.customer_name
                        ORDER BY total_sales_2024 DESC""")

print(cursor.fetchall())

[(3, 'Charlie', 350.0), (1, 'Alice', 200.5), (2, 'Bob', 150.85)]


In [11]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS Customer (
    customer_id INTEGER PRIMARY KEY,
    customer_name TEXT)""")

In [14]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS Orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date DATE,
    total_amount REAL,
    FOREIGN KEY (customer_id) REFERENCES Customer(customer_id))""")

In [39]:
cursor.executemany("""
INSERT OR REPLACE INTO Customer (customer_id, customer_name)
VALUES(?,?)
""", [
    (1, "Alice"),
    (2, "Bob"),
    (3, "Charlie")
])

In [38]:
cursor.executemany("""
INSERT OR REPLACE INTO Orders (order_id, customer_id, order_date, total_amount)
VALUES (?,?,?,?)
""", [
    (1,1,'2024-02-15',200.50),
    (2,2,'2025-01-01',150.85),
    (3,3,'2024-8-20', 350.00),
    (4,4,'01-02-2001',100.50)
])

conn.commit()

In [29]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('Customer',), ('Orders',)]


In [32]:
cursor.execute("PRAGMA table_info (Customer)")
print(cursor.fetchall())

[(0, 'customer_id', 'INTEGER', 0, None, 1), (1, 'customer_name', 'TEXT', 0, None, 0)]


In [37]:
cursor.execute("SELECT * FROM Orders LIMIT 4")
print(cursor.fetchall())

[(1, 1, '2024-02-15', 200.5), (2, 2, '2025-01-01', 150.85), (3, 3, '2024-8-20', 350.0), (4, 4, '01-02-2001', 100.5)]


In [43]:
cursor.execute("SELECT COUNT(*) FROM Customer;")
print(cursor.fetchall())

[(3,)]


In [45]:
def selectData(students: pd.DataFrame) -> pd.DataFrame:
    return students.loc[students['student_id'] == 101, ['name','age']]

In [46]:
def createBonusColumn(students: pd.DataFrame) -> pd.DataFrame:
    students['bonus'] = students['salary'] * 2
    return students

In [56]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS Employee (
    employee_id INTEGER PRIMARY KEY,
    employee_name TEXT,
    experience_years REAL)""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Project (
    project_id INTEGER PRIMARY KEY,
    project_name TEXT,
    employee_id INTEGER,
    FOREIGN KEY (employee_id) REFERENCES Employee(employee_id))""")

cursor.executemany("""
INSERT OR REPLACE INTO Employee (employee_id, employee_name, experience_years)
VALUES (?, ?, ?)
""", [
    (1, "Alice", 5),
    (2, "Bob", 8),
    (3, "Charlie", 3)
])

cursor.executemany("""
INSERT OR REPLACE INTO Project (project_id, project_name, employee_id)
VALUES (?, ?, ?)
""", [
    (101, "AI System", 1),
    (102, "Data Pipeline", 2),
    (103, "Web App", 3),
    (104, "Analytics Dashboard", 2)
])

conn.commit()


cursor.execute("""
SELECT p.project_id,
ROUND(AVG(e.experience_years),2) AS average_years
FROM Project p
JOIN Employee e
ON p.employee_id = e.employee_id
GROUP BY p.project_id""")

print("Database query result:\n", cursor.fetchall())

Database query result:
 [(101, 5.0), (102, 8.0), (103, 3.0), (104, 8.0)]


In [61]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS Queries (
    query_name TEXT,
    rating INTEGER,
    position INTEGER)""")

cursor.executemany("""
INSERT INTO Queries (query_name, rating, position)
VALUES (?, ?, ?)
""", [
    ("query1", 4, 1),
    ("query1", 2, 2),
    ("query2", 5, 1),
    ("query2", 1, 3),
    ("query3", 3, 2)
])

cursor.execute("""
SELECT query_name,
ROUND(AVG(rating/position),2) AS quality,
ROUND(AVG(rating<3)*100,2) AS poor_query_percentage
FROM Queries
GROUP BY query_name""")

cursor.fetchall()

[('query1', 2.5, 50.0), ('query2', 2.5, 50.0), ('query3', 1.0, 0.0)]